In [2]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

FINAL_DATA_NAME = 'approx1e5-agriVision-fourier' # + channel
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "agriVision-RGB-cleaned"
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'raw-data','agriVision')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir
BATCH_NUM = 0

In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))

In [4]:
file_list = [os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}"))
file_names[:5]

['.DS_Store', 'batch 1', 'batch 3', 'batch 2']

In [5]:
'''Assuming No batching is required. Not applicable for agriVision'''

# data_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", "full-agriVision-RGB-cleaned")

# for channel in ['red', 'blue', 'green', 'gray', 'infrared']:

#     channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True)
#     pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-agriVision-fourier-{channel}-df.pickle"))

#     min_group, max_group = 2, max(channel_fr['band'])
#     group_data_map = dict()
#     group_data_map_size = dict()
#     for group in np.arange(min_group, max_group + 1):
#         data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
#         group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
#         group_data_map_size[group] = data.size
    
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

'Assuming No batching is required. Not applicable for agriVision'

In [6]:
'''To split large dataset into many batches, only needs to be run once'''
# k = 10000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'agriVision') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

'To split large dataset into many batches, only needs to be run once'

In [7]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))


batch1-agriVision-RGB-cleaned
batch2-agriVision-RGB-cleaned
toy-agriVision-RGB-cleaned
batch0-agriVision-RGB-cleaned
full-agriVision-RGB-cleaned



# Fourier

In [7]:
#Values obtained from plots in agriVisionFourierEDA.ipynb
batch_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")
splits = getSplits(0.008052940675034493,0.5282905274067067, 1.15)
print(splits)

[0.008052940675034493, 0.009260881776289667, 0.010650014042733115, 0.012247516149143082, 0.014084643571514543, 0.016197340107241723, 0.01862694112332798, 0.021420982291827175, 0.02463412963560125, 0.028329249080941435, 0.03257863644308265, 0.037465431909545044, 0.0430852466959768, 0.04954803370037331, 0.056980238755429305, 0.0655272745687437, 0.07535636575405526, 0.08665982061716354, 0.09965879370973807, 0.11460761276619877, 0.13179875468112856, 0.15156856788329784, 0.1743038530657925, 0.20044943102566135, 0.23051684567951053, 0.2650943725314371, 0.30485852841115263, 0.3505873076728255, 0.4031754038237493, 0.4636517143973116]


In [ ]:
channel = "red"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4500/4500 [03:15<00:00, 23.07it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171]


100%|██████████| 30/30 [00:40<00:00,  1.33s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,red,"(0.0, 0.0078125)",10,"[-118092.0, -209566.81, 80184.59, -10856.434, ..."
1,2,red,"(0.008052940675034493, 0.008734640537108554)",3,"[-2029.8997, -180.56264, 2404.6543, -2978.5793..."
2,3,red,"(0.009765625, 0.010517900013934578)",3,"[7238.685, 8688.75, -69.16031, -8039.517, -219..."
3,4,red,"(0.011048543456039806, 0.01188039556698871)",4,"[9876.31, 5580.1226, -927.84576, 1846.173, 292..."
4,5,red,"(0.012352647110032733, 0.014084184669781208)",6,"[-289.08902, -4747.2026, 10580.036, 2368.381, ..."


In [9]:
del channel_fr, group_data_map, group_data_map_size

In [ ]:
channel = "blue"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4500/4500 [03:05<00:00, 24.24it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171]


100%|██████████| 30/30 [00:37<00:00,  1.26s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,blue,"(0.0, 0.0078125)",10,"[-170415.45, 510926.2, 122433.34, -178011.03, ..."
1,2,blue,"(0.008052940675034493, 0.008734640537108554)",3,"[804.20624, -3518.8442, 5547.617, -132.90826, ..."
2,3,blue,"(0.009765625, 0.010517900013934578)",3,"[1635.7427, -4306.875, -5431.872, -2827.784, -..."
3,4,blue,"(0.011048543456039806, 0.01188039556698871)",4,"[1375.6718, 4996.138, 7108.4253, -97.149925, -..."
4,5,blue,"(0.012352647110032733, 0.014084184669781208)",6,"[-94.99612, -2011.3475, -20544.59, -951.52094,..."


In [11]:
del channel_fr, group_data_map, group_data_map_size

In [ ]:
channel = "green"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4500/4500 [03:15<00:00, 22.96it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171]


100%|██████████| 30/30 [00:43<00:00,  1.43s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,green,"(0.0, 0.0078125)",10,"[-148763.56, 211995.89, 128138.3, -143010.67, ..."
1,2,green,"(0.008052940675034493, 0.008734640537108554)",3,"[-89.69858, 34.625412, 3640.0142, 4140.314, 10..."
2,3,green,"(0.009765625, 0.010517900013934578)",3,"[982.6647, -19958.291, -2931.4756, -3953.9229,..."
3,4,green,"(0.011048543456039806, 0.01188039556698871)",4,"[710.74634, 10861.79, 10518.6875, -5391.78, -8..."
4,5,green,"(0.012352647110032733, 0.014084184669781208)",6,"[-307.9739, 4390.76, -37273.09, -2989.3308, 12..."


In [13]:
del channel_fr, group_data_map, group_data_map_size

In [ ]:
channel = "gray"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4500/4500 [03:48<00:00, 19.71it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171]


100%|██████████| 30/30 [00:45<00:00,  1.52s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,gray,"(0.0, 0.0078125)",10,"[-142049.27, 120047.66, 113141.76, -107485.51,..."
1,2,gray,"(0.008052940675034493, 0.008734640537108554)",3,"[-567.7106, -434.79327, 3487.868, 1524.9153, 8..."
2,3,gray,"(0.009765625, 0.010517900013934578)",3,"[2926.942, -9609.433, -2360.6816, -5046.3315, ..."
3,4,gray,"(0.011048543456039806, 0.01188039556698871)",4,"[3526.0637, 8613.329, 6707.497, -2624.2285, -5..."
4,5,gray,"(0.012352647110032733, 0.014084184669781208)",6,"[-278.01895, 929.14343, -21059.014, -1155.3014..."


In [15]:
del channel_fr, group_data_map, group_data_map_size

In [ ]:
channel = "infrared"

channel_fr = convert_to_fourier_basis(batch_dir, channel, split_list = splits, debug = True, image_opener = npz_opener)
channel_fr['data'] = channel_fr['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_fr['band'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_fr.head()

100%|██████████| 4500/4500 [03:54<00:00, 19.18it/s]


[0.00805294 0.00926088 0.01065001 0.01224752 0.01408464 0.01619734
 0.01862694 0.02142098 0.02463413 0.02832925 0.03257864 0.03746543
 0.04308525 0.04954803 0.05698024 0.06552727 0.07535637 0.08665982
 0.09965879 0.11460761 0.13179875 0.15156857 0.17430385 0.20044943
 0.23051685 0.26509437 0.30485853 0.35058731 0.4031754  0.46365171]


100%|██████████| 30/30 [00:40<00:00,  1.36s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,infrared,"(0.0, 0.0078125)",10,"[-142049.27, 120047.66, 113141.76, -107485.51,..."
1,2,infrared,"(0.008052940675034493, 0.008734640537108554)",3,"[-567.7106, -434.79327, 3487.868, 1524.9153, 8..."
2,3,infrared,"(0.009765625, 0.010517900013934578)",3,"[2926.942, -9609.433, -2360.6816, -5046.3315, ..."
3,4,infrared,"(0.011048543456039806, 0.01188039556698871)",4,"[3526.0637, 8613.329, 6707.497, -2624.2285, -5..."
4,5,infrared,"(0.012352647110032733, 0.014084184669781208)",6,"[-278.01895, 929.14343, -21059.014, -1155.3014..."


In [17]:
del channel_fr, group_data_map, group_data_map_size

# Wavelet

In [9]:
FINAL_DATA_NAME = 'approx1e5-agriVision-wavelet'
batch_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [8]:
channel = "red"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0]).astype(int)
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"test_int_batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


100%|██████████| 4500/4500 [02:19<00:00, 32.15it/s]


,channel,layer,orientation,data
0,red,1,L1,"[12.37854, 388.76587, -71.011154, -525.40607, ..."
1,red,2,D,"[-56.85054, 10.112775, -65.61645, 3.6756237, -..."
2,red,2,H,"[426.77856, 36.807053, 2.2234993, 41.115826, 3..."
3,red,2,V,"[-458.58713, 52.57764, -45.266056, -4.2271876,..."
4,red,3,D,"[-192.24008, 1.2320688, -6.463406, 114.53845, ..."


In [9]:
del channel_wv, group_data_map, group_data_map_size

In [10]:
channel = "green"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))

min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


100%|██████████| 4500/4500 [02:30<00:00, 29.97it/s]


,channel,layer,orientation,data
0,green,1,L1,"[166.38116, -696.27155, -199.10657, -450.9094,..."
1,green,2,D,"[-78.65823, 1.7065886, -69.58261, 4.20013, 16...."
2,green,2,H,"[470.16617, 13.996262, 113.74389, 31.502106, -..."
3,green,2,V,"[-340.02054, 20.371666, -35.55738, 18.073538, ..."
4,green,3,D,"[-147.98283, 9.817328, -40.521946, 67.97765, -..."


In [11]:
del channel_wv, group_data_map, group_data_map_size

In [12]:
channel = "blue"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


100%|██████████| 4500/4500 [02:28<00:00, 30.40it/s]


,channel,layer,orientation,data
0,blue,1,L1,"[394.00897, -666.8051, -88.63963, -346.68665, ..."
1,blue,2,D,"[-79.15217, 5.5673375, -78.97526, 2.18005, 15...."
2,blue,2,H,"[493.76672, 17.22162, 125.757225, 35.976807, -..."
3,blue,2,V,"[-525.7132, 29.988464, -39.572536, 13.509995, ..."
4,blue,3,D,"[-235.7314, -6.8745494, -2.5346868, 127.94007,..."


In [1]:
del channel_wv, group_data_map, group_data_map_size

NameError: name 'channel_wv' is not defined

In [10]:
channel = "gray"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


100%|██████████| 4500/4500 [02:52<00:00, 26.10it/s]


,channel,layer,orientation,data
0,gray,1,L1,"[146.28271, -368.52505, -148.2057, -461.24997,..."
1,gray,2,D,"[-72.188354, 4.6591525, -69.46092, 3.812646, 7..."
2,gray,2,H,"[459.84106, 21.180698, 81.76859, 34.882614, 7...."
3,gray,2,V,"[-396.59506, 31.09231, -38.913475, 10.885799, ..."
4,gray,3,D,"[-171.19986, 5.347338, -26.007248, 88.723595, ..."


In [11]:
del channel_wv, group_data_map, group_data_map_size

In [ ]:
channel = "infrared"

channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug = True, image_opener = npz_opener)
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
if SAVE_DF:
    pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-df.pickle"))


min_group, max_group = 2, max(channel_wv['layer'])
group_data_map = dict()
group_data_map_size = dict()
for group in np.arange(min_group, max_group + 1):
    data = np.append(channel_wv[(channel_wv['orientation'] == 'H') & (channel_wv['layer'] == group)]['data'].iloc[0],
                     channel_wv[(channel_wv['orientation'] == 'V') & (channel_wv['layer'] == group)]['data'].iloc[0])
    group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
    group_data_map_size[group] = data.size
    
pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}.pickle"))
pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME}-{channel}-size.pickle"))

channel_wv.head()

10 layers being used


100%|██████████| 4500/4500 [07:54<00:00,  9.48it/s]


,channel,layer,orientation,data
0,infrared,1,L1,"[-277.5, 234.5, 221.0, -209.9, 523.5, -94.3, -..."
1,infrared,2,D,"[-32.28, 12.57, -84.2, -12.9, -55.62, 152.6, -..."
2,infrared,2,H,"[-157.0, 88.5, -302.5, 103.94, 9.93, 7.09, -2...."
3,infrared,2,V,"[74.4, 56.47, -52.66, -119.2, 14.516, 47.44, 2..."
4,infrared,3,D,"[-9.8, -17.81, -62.2, 17.61, 21.64, -0.794, -3..."


In [ ]:
del channel_wv, group_data_map, group_data_map_size